# Testing

In this notebook you can explore and test the resulting database tables from our Dagster ETL process.
Here you can analyze


In [2]:
import duckdb
import polars as pl

# from IPython import display
import geopandas as gpd
from shapely import wkt

### Create Database Connection


In [3]:
%load_ext sql
conn = duckdb.connect(database="../dsp-dagster/data_systems_project.duckdb")
%sql conn --alias duckdb

Loading configurations from /home/workspaces/DSP-compose/pyproject.toml.

Settings changed:

Config,value
displaycon,False
feedback,True
autopandas,True


In [4]:
%sql SHOW ALL TABLES; # shows all available tables

,database,schema,name,column_names,column_types,temporary
0,data_systems_project,public,cbs_buurten,"[geometry, buurtcode, buurtnaam, wijkcode, gem...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
1,data_systems_project,public,cbs_wijken,"[geometry, wijkcode, wijknaam, gemeentecode, g...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
2,data_systems_project,public,fire_stations_and_vehicles,"[Fire_Station, Vehicle, Vehicle_Type]","[VARCHAR, VARCHAR, VARCHAR]",False
3,data_systems_project,public,service_areas,"[H_Verzorgingsgebied_ID, Verzorgingsgebied, LA...","[BIGINT, VARCHAR, DOUBLE, DOUBLE, VARCHAR]",False
4,data_systems_project,public,storm_deployments,"[Deployment_ID, Incident_ID, Vehicle_Type, Veh...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, VA...",False
5,data_systems_project,public,storm_incidents,"[Incident_ID, Date, Incident_Starttime, Incide...","[BIGINT, TIMESTAMP_MS, TIME, TIME, TIME, DOUBL...",False


##### Drop Tables


In [ ]:
# %sql DROP TABLE joined.incident_deployments_vehicles_weather;
# %sql DROP TABLE joined.incident_deployments_vehicles_wijken;
# %sql DROP TABLE joined.incident_deployments_vehicles;
# %sql DROP TABLE joined.incidents_buurten;
%sql DROP TABLE joined.knmi_weather_txt;
# %sql DROP TABLE joined.buurten_trees;
# %sql DROP TABLE cleaned.cleaned_storm_incidents;
# %sql DROP TABLE joined.buurten_incidents_trees;

In [5]:
buurten_incidents_trees = conn.execute(
    """
    SELECT * FROM joined.buurten_incidents_trees
    """
).pl()


# Close the database connection
conn.close()

CatalogException: Catalog Error: Table with name buurten_incidents_trees does not exist!
Did you mean "buurten_incidents"?
LINE 2:     SELECT * FROM joined.buurten_incidents_trees
                          ^

In [6]:
def convert_to_geodf(polars_df: pl.DataFrame) -> gpd.GeoDataFrame:
    """
    Convert a Polars DataFrame to a GeoDataFrame using WKB or WKT transformation.
    """

    # Convert Polars DataFrame to Pandas DataFrame
    df = polars_df.to_pandas()

    # Convert geometry strings back to geometry objects
    if "geometry" in df.columns:
        df["geometry"] = df["geometry"].apply(wkt.loads)

    else:
        raise ValueError("No 'geometry' column found in the DataFrame")

    # Convert back to GeoDataFrame
    return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")


def convert_to_polars(gdf: gpd.GeoDataFrame) -> pl.DataFrame:
    """
    Convert a GeoDataFrame to a Polars DataFrame, converting geometries to WKB strings.
    """
    # If geometry conversion is necessary, uncomment the following line
    # gdf["geometry"] = gdf["geometry"].apply(lambda geom: wkb_dumps(geom, hex=True))
    gdf["geometry"] = gdf["geometry"].apply(wkt.dumps)

    # Convert to Polars DataFrame
    return pl.from_pandas(gdf)

### Buurten Incidents


In [7]:
print(buurten_incidents.columns)
buurten_incidents.head()

NameError: name 'buurten_incidents' is not defined

In [ ]:
df = buurten_incidents.group_by(
    [
        "buurtcode",
        "Date",
        "Incident_Starttime_Hour",
        "Damage_Type",
    ]
).agg(pl.col("Incident_ID").count().alias("Totaal"))

### Importing dataframes from parquet

In [8]:
import pandas as pd

buurten_combined = pl.read_parquet("../dsp-dagster/data/buurten_combined.parquet")
buurten_combined

geometry,buurtcode,buurtnaam,wijkcode,gemeentecode,gemeentenaam,indelingswijzigingWijkenEnBuurten,water,meestVoorkomendePostcode,dekkingspercentage,omgevingsadressendichtheid,stedelijkheidAdressenPerKm2,bevolkingsdichtheidInwonersPerKm2,aantalInwoners,mannen,vrouwen,percentagePersonen0Tot15Jaar,percentagePersonen15Tot25Jaar,percentagePersonen25Tot45Jaar,percentagePersonen45Tot65Jaar,percentagePersonen65JaarEnOuder,percentageOngehuwd,percentageGehuwd,percentageGescheid,percentageVerweduwd,aantalHuishoudens,percentageEenpersoonshuishoudens,percentageHuishoudensZonderKinderen,percentageHuishoudensMetKinderen,gemiddeldeHuishoudsgrootte,percentageWesterseMigratieachtergrond,percentageNietWesterseMigratieachtergrond,percentageUitMarokko,percentageUitNederlandseAntillenEnAruba,percentageUitSuriname,percentageUitTurkije,percentageOverigeNietwestersemigratieachtergrond,…,Trees_soortnaamTop_Albizia,Trees_soortnaamTop_Tetracentron,Trees_soortnaamTop_Oxydendrum,Trees_soortnaamTop_Cladrastis,Trees_standplaatsGedetailleerd_Straatbaksteen,Trees_standplaatsGedetailleerd_Lijnvormige-haag,Trees_standplaatsGedetailleerd_Gazon,Trees_standplaatsGedetailleerd_Tegels,Trees_standplaatsGedetailleerd_Bomen-en-struikvormers,Trees_standplaatsGedetailleerd_Fijne-sierheester,Trees_standplaatsGedetailleerd_Vaste-planten,Trees_standplaatsGedetailleerd_Struikvormers,Trees_standplaatsGedetailleerd_Betonstraatstenen,Trees_standplaatsGedetailleerd_Ruw-gras,Trees_standplaatsGedetailleerd_Bloemrijk-gras,Trees_standplaatsGedetailleerd_Bodembedekkende-heesters,Trees_standplaatsGedetailleerd_Cultuurrozen,Trees_standplaatsGedetailleerd_Ruigte,Trees_standplaatsGedetailleerd_Los,Trees_standplaatsGedetailleerd_Sierbestrating,Trees_standplaatsGedetailleerd_Samenhangend,Trees_standplaatsGedetailleerd_Boomvormers,Trees_standplaatsGedetailleerd_Dichte-deklagen,Trees_standplaatsGedetailleerd_Ongewapend-nietverdeuveld-beton,Trees_standplaatsGedetailleerd_Blokhaag,Trees_standplaatsGedetailleerd_Grove-sierheester,Trees_standplaatsGedetailleerd_Heesterrozen,Trees_standplaatsGedetailleerd_Botanische-rozen,"Trees_stamdiameterklasse_0,1-tot-0,2-m.","Trees_stamdiameterklasse_0,5-tot-1-m.","Trees_stamdiameterklasse_0,3-tot-0,5-m.","Trees_stamdiameterklasse_0,2-tot-0,3-m.","Trees_stamdiameterklasse_1,0-tot-1,5-m.",Incidents_Date,Incidents_Incident_Starttime_Hour,Incidents_Damage_Type,Incidents_Total
str,str,str,str,str,str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,datetime[ms],i8,str,u32
"""MULTIPOLYGON (…","""BU04792130""","""Het Eiland""","""WK047921""","""GM0479""","""Zaanstad""",1,"""NEE""","""1506""",1,3064,1,9972,1265,630,635,13,14,13,33,27,41,46,7,7,540,27,34,39,2.3,12,13,1,1,4,3,5,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""MULTIPOLYGON (…","""BU04796120""","""Noorderhoofdbu…","""WK047961""","""GM0479""","""Zaanstad""",1,"""NEE""","""1561""",1,2171,2,5036,2040,1020,1020,13,8,26,28,26,48,33,12,7,1140,52,24,24,1.8,8,12,2,0,2,3,4,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""MULTIPOLYGON (…","""BU03620401""","""Stadshart""","""WK036204""","""GM0362""","""Amstelveen""",1,"""NEE""","""1181""",1,3693,1,6297,3480,1780,1700,17,7,36,23,18,47,43,6,4,1780,47,25,29,2.0,31,37,1,1,2,1,32,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-03-13 00:00:00,15,"""Building""",1
"""MULTIPOLYGON (…","""BU03620401""","""Stadshart""","""WK036204""","""GM0362""","""Amstelveen""",1,"""NEE

In [9]:
buurten_combined = buurten_combined.with_columns(pl.col("Incidents_Date").cast(pl.Date))
buurten_combined

geometry,buurtcode,buurtnaam,wijkcode,gemeentecode,gemeentenaam,indelingswijzigingWijkenEnBuurten,water,meestVoorkomendePostcode,dekkingspercentage,omgevingsadressendichtheid,stedelijkheidAdressenPerKm2,bevolkingsdichtheidInwonersPerKm2,aantalInwoners,mannen,vrouwen,percentagePersonen0Tot15Jaar,percentagePersonen15Tot25Jaar,percentagePersonen25Tot45Jaar,percentagePersonen45Tot65Jaar,percentagePersonen65JaarEnOuder,percentageOngehuwd,percentageGehuwd,percentageGescheid,percentageVerweduwd,aantalHuishoudens,percentageEenpersoonshuishoudens,percentageHuishoudensZonderKinderen,percentageHuishoudensMetKinderen,gemiddeldeHuishoudsgrootte,percentageWesterseMigratieachtergrond,percentageNietWesterseMigratieachtergrond,percentageUitMarokko,percentageUitNederlandseAntillenEnAruba,percentageUitSuriname,percentageUitTurkije,percentageOverigeNietwestersemigratieachtergrond,…,Trees_soortnaamTop_Albizia,Trees_soortnaamTop_Tetracentron,Trees_soortnaamTop_Oxydendrum,Trees_soortnaamTop_Cladrastis,Trees_standplaatsGedetailleerd_Straatbaksteen,Trees_standplaatsGedetailleerd_Lijnvormige-haag,Trees_standplaatsGedetailleerd_Gazon,Trees_standplaatsGedetailleerd_Tegels,Trees_standplaatsGedetailleerd_Bomen-en-struikvormers,Trees_standplaatsGedetailleerd_Fijne-sierheester,Trees_standplaatsGedetailleerd_Vaste-planten,Trees_standplaatsGedetailleerd_Struikvormers,Trees_standplaatsGedetailleerd_Betonstraatstenen,Trees_standplaatsGedetailleerd_Ruw-gras,Trees_standplaatsGedetailleerd_Bloemrijk-gras,Trees_standplaatsGedetailleerd_Bodembedekkende-heesters,Trees_standplaatsGedetailleerd_Cultuurrozen,Trees_standplaatsGedetailleerd_Ruigte,Trees_standplaatsGedetailleerd_Los,Trees_standplaatsGedetailleerd_Sierbestrating,Trees_standplaatsGedetailleerd_Samenhangend,Trees_standplaatsGedetailleerd_Boomvormers,Trees_standplaatsGedetailleerd_Dichte-deklagen,Trees_standplaatsGedetailleerd_Ongewapend-nietverdeuveld-beton,Trees_standplaatsGedetailleerd_Blokhaag,Trees_standplaatsGedetailleerd_Grove-sierheester,Trees_standplaatsGedetailleerd_Heesterrozen,Trees_standplaatsGedetailleerd_Botanische-rozen,"Trees_stamdiameterklasse_0,1-tot-0,2-m.","Trees_stamdiameterklasse_0,5-tot-1-m.","Trees_stamdiameterklasse_0,3-tot-0,5-m.","Trees_stamdiameterklasse_0,2-tot-0,3-m.","Trees_stamdiameterklasse_1,0-tot-1,5-m.",Incidents_Date,Incidents_Incident_Starttime_Hour,Incidents_Damage_Type,Incidents_Total
str,str,str,str,str,str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,date,i8,str,u32
"""MULTIPOLYGON (…","""BU04792130""","""Het Eiland""","""WK047921""","""GM0479""","""Zaanstad""",1,"""NEE""","""1506""",1,3064,1,9972,1265,630,635,13,14,13,33,27,41,46,7,7,540,27,34,39,2.3,12,13,1,1,4,3,5,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""MULTIPOLYGON (…","""BU04796120""","""Noorderhoofdbu…","""WK047961""","""GM0479""","""Zaanstad""",1,"""NEE""","""1561""",1,2171,2,5036,2040,1020,1020,13,8,26,28,26,48,33,12,7,1140,52,24,24,1.8,8,12,2,0,2,3,4,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""MULTIPOLYGON (…","""BU03620401""","""Stadshart""","""WK036204""","""GM0362""","""Amstelveen""",1,"""NEE""","""1181""",1,3693,1,6297,3480,1780,1700,17,7,36,23,18,47,43,6,4,1780,47,25,29,2.0,31,37,1,1,2,1,32,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-03-13,15,"""Building""",1
"""MULTIPOLYGON (…","""BU03620401""","""Stadshart""","""WK036204""","""GM0362""","""Amstelveen""",1,"""NEE""","""1181""",1,

In [10]:
knmi_weather_data = pl.read_parquet("../dsp-dagster/data/knmi_weather_data.parquet")
knmi_weather_data

STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,SQ,Q,DR,RH,P,VV,N,U,WW,IX,M,R,S,O,Y
i64,date,i8,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64
240,2024-01-22,18,240,100,100,160,85,""" 76""",48,0,0,0,0,10099,65,0,77,""" """,7,0,0,0,0,0
240,2024-01-22,17,240,100,100,160,87,""" """,44,0,0,0,0,10090,66,1,74,""" """,5,0,0,0,0,0
240,2024-01-22,16,240,110,110,160,90,""" """,48,3,13,0,0,10085,65,6,75,""" """,5,0,0,0,0,0
240,2024-01-22,15,240,120,120,190,94,""" """,50,10,43,0,0,10073,64,2,74,""" """,5,0,0,0,0,0
240,2024-01-22,14,240,130,130,190,98,""" """,52,10,79,0,0,10069,66,5,73,""" """,5,0,0,0,0,0
240,2024-01-22,13,240,120,120,200,100,""" """,57,10,97,0,0,10065,64,1,74,""" """,5,0,0,0,0,0
240,2024-01-22,12,250,120,120,180,104,""" 89""",63,10,101,0,0,10060,65,0,76,""" 2""",7,0,0,0,0,0
240,2024-01-22,11,250,120,130,190,101,""" """,71,10,82,0,0,10050,63,1,81,""" """,7,0,0,0,0,0
240,2024-01-22,10,240,100,100,160,96,""" """,73,0,18,0,-1,10042,60,7,85,""" 23""",7,0,1,0,0,0


In [11]:
# Perform the join without equality conditions on date and hour
model_data = buurten_combined.join(
    knmi_weather_data,
    left_on=("Incidents_Date", "Incidents_Incident_Starttime_Hour"),
    right_on=("YYYYMMDD", "HH"),
)
model_data

geometry,buurtcode,buurtnaam,wijkcode,gemeentecode,gemeentenaam,indelingswijzigingWijkenEnBuurten,water,meestVoorkomendePostcode,dekkingspercentage,omgevingsadressendichtheid,stedelijkheidAdressenPerKm2,bevolkingsdichtheidInwonersPerKm2,aantalInwoners,mannen,vrouwen,percentagePersonen0Tot15Jaar,percentagePersonen15Tot25Jaar,percentagePersonen25Tot45Jaar,percentagePersonen45Tot65Jaar,percentagePersonen65JaarEnOuder,percentageOngehuwd,percentageGehuwd,percentageGescheid,percentageVerweduwd,aantalHuishoudens,percentageEenpersoonshuishoudens,percentageHuishoudensZonderKinderen,percentageHuishoudensMetKinderen,gemiddeldeHuishoudsgrootte,percentageWesterseMigratieachtergrond,percentageNietWesterseMigratieachtergrond,percentageUitMarokko,percentageUitNederlandseAntillenEnAruba,percentageUitSuriname,percentageUitTurkije,percentageOverigeNietwestersemigratieachtergrond,…,Trees_standplaatsGedetailleerd_Ongewapend-nietverdeuveld-beton,Trees_standplaatsGedetailleerd_Blokhaag,Trees_standplaatsGedetailleerd_Grove-sierheester,Trees_standplaatsGedetailleerd_Heesterrozen,Trees_standplaatsGedetailleerd_Botanische-rozen,"Trees_stamdiameterklasse_0,1-tot-0,2-m.","Trees_stamdiameterklasse_0,5-tot-1-m.","Trees_stamdiameterklasse_0,3-tot-0,5-m.","Trees_stamdiameterklasse_0,2-tot-0,3-m.","Trees_stamdiameterklasse_1,0-tot-1,5-m.",Incidents_Date,Incidents_Incident_Starttime_Hour,Incidents_Damage_Type,Incidents_Total,STN,DD,FH,FF,FX,T,T10N,TD,SQ,Q,DR,RH,P,VV,N,U,WW,IX,M,R,S,O,Y
str,str,str,str,str,str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,date,i8,str,u32,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64
"""MULTIPOLYGON (…","""BU03630402""","""Rapenburg""","""WK036304""","""GM0363""","""Amsterdam""",1,"""NEE""","""1011""",1,6056,1,10753,990,480,505,9,22,36,22,11,72,21,6,1,630,60,25,14,1.6,34,17,1,1,3,2,11,…,null,null,null,null,null,null,null,null,null,null,2023-11-02,18,"""Unknown""",1,240,190,150,150,230,109,""" 94""",62,0,0,0,-1,9709,75,8,72,""" 23""",7,0,1,0,0,0
"""MULTIPOLYGON (…","""BU03638802""","""Nieuwe Meer""","""WK036388""","""GM0363""","""Amsterdam""",1,"""NEE""","""1066""",1,271,5,137,175,100,80,12,15,22,44,8,76,16,7,1,115,71,5,24,1.6,22,7,0,2,1,0,5,…,null,null,null,null,null,null,null,null,null,null,2023-11-02,18,"""Tree""",1,240,190,150,150,230,109,""" 94""",62,0,0,0,-1,9709,75,8,72,""" 23""",7,0,1,0,0,0
"""MULTIPOLYGON (…","""BU03620202""","""Patrimonium""","""WK036202""","""GM0362""","""Amstelveen""",1,"""NEE""","""1182""",1,1462,3,4490,2220,1070,1145,14,11,20,29,26,48,38,9,5,1080,41,25,33,2.1,19,16,2,1,2,0,11,…,null,null,null,null,null,null,null,null,null,null,2023-11-02,18,"""Tree""",1,240,190,150,150,230,109,""" 94""",62,0,0,0,-1,9709,75,8,72,""" 23""",7,0,1,0,0,0
"""MULTIPOLYGON (…","""BU03630907""","""Kazernebuurt""","""WK036309""","""GM0363""","""Amsterdam""",1,"""NEE""","""1018""",1,8035,1,16958,930,440,485,6,10,34,26,24,71,18,9,3,620,67,21,13,1.5,20,21,4,1,5,2,10,…,null,null,null,null,null,null,null,null,null,null,2023-11-02,18,"""Unknown""",1,240,190,150,150,230,109,""" 94""",62,0,0,0,-1,9709,75,8,72,""" 23""",7,0,1,0,0,0
"""MULTIPOLYGON (…","""BU03635202""","""Scheldebuurt M…","""WK036352""","""GM0363""","""Amsterdam""",1,"""NEE""","""1078""",3,8568,1,19667,3150,1440,1710,14,10,42,19,15,67,23,7,3,1665,46,30,24,1.9,25,11,1,1,2,1,6,…,0,0,0,0,0,0,0,0,0,0,2023-11-02,18,"""Tree""",1,240,190,150,150,230,109,""" 94""",62,0,0,0,-1,9709,75,8,72,""" 23""",7,0,1,0,0,0
"""MULTIPOLYGON (…","""BU03638903""","""Medisch Centru…","""WK036389""","""GM0363""","""Amsterdam""",1,"""NEE""","""1066""",1,2509,1,2748,795,410,385,6,46,35,6,6,86,5,7,2,650,91,3,6,1.2,27,40,11,2,5,3,19,…,null,null,null,null,null,null,null,null,null,null,2023-11-02,18,"""Tree""",1,240,190,150,150,230,109,""" 94""",62,0,0,0,-1,9709,75,8,72,""" 23""",7,0,1,0,0,0
"""MULTIPOLYGON (…","""BU03635602""","""Middenme

In [12]:
model_data.write_parquet("model_data.parquet")

In [15]:
model_data["buurtcode"].unique()

buurtcode
str
"""BU03630105"""
"""BU03638805"""
"""BU03630408"""
"""BU03630600"""
"""BU03639200"""
"""BU03631401"""
"""BU03633201"""
"""BU03621104"""
"""BU03635504"""
